In [4]:
import ast
import glob
import warnings
from collections import defaultdict
from datetime import date

import numpy as np
import pandas as pd
import wandb

today = date.today()
api = wandb.Api()

# # Find all csv files in the current directory
csv_files = glob.glob("/home/lev/projects/TopoBenchmarkX/big_csv/*.csv")
# # Collect all the names of the csv files without the extension
csv_names = [csv_file[:-4] for csv_file in csv_files]
project_name = "TopoBenchmarkX_Simplicial"  
user = "telyatnikov_sap"

if project_name not in csv_names:
    runs = api.runs(f"{user}/{project_name}")

    summary_list, config_list, name_list = [], [], []
    for run in runs:
        # .summary contains the output keys/values for metrics like accuracy.
        #  We call ._json_dict to omit large files
        summary_list.append(run.summary._json_dict)

        # .config contains the hyperparameters.
        #  We remove special values that start with _.
        config_list.append(
            {k: v for k, v in run.config.items() if not k.startswith("_")}
        )

        # .name is the human-readable name of the run.
        name_list.append(run.name)

    runs_df = pd.DataFrame(
        {"summary": summary_list, "config": config_list, "name": name_list}
    )

    runs_df.to_csv(f"{project_name}.csv")
else:
    runs_df = pd.read_csv(f"{project_name}.csv", index_col=0)

    for row in runs_df.iloc:
        row["summary"] = ast.literal_eval(row["summary"])
        row["config"] = ast.literal_eval(row["config"])


for row in runs_df.iloc:
    row["summary"].update(row["config"])

lst = [i["summary"] for i in runs_df.iloc]
df = pd.DataFrame.from_dict(lst)

df_init = df.copy()

# Get average epoch run time
df["epoch_run_time"] = df["_runtime"] / df["epoch"]

In [5]:
def normalize_column(df, column_to_normalize):
    # Use json_normalize to flatten the nested dictionaries into separate columns
    flattened_df = pd.json_normalize(df[column_to_normalize])
    # Rename columns to include 'nested_column' prefix
    flattened_df.columns = [
        f"{column_to_normalize}.{col}" for col in flattened_df.columns
    ]
    # Concatenate the flattened DataFrame with the original DataFrame
    result_df = pd.concat([df, flattened_df], axis=1)
    # Get new columns names
    new_columns = flattened_df.columns
    # Drop the original nested column if needed
    result_df.drop(column_to_normalize, axis=1, inplace=True)
    return result_df, new_columns


# Config columns to normalize
columns_to_normalize = ["model", "dataset", "callbacks", "paths"]

# Keep track of config columns added
config_columns = []
for column in columns_to_normalize:
    df, columns = normalize_column(df, column)
    config_columns.extend(columns)

## Select models that have finished the runs

## Workout us_demographic 

In [6]:
# For every rows where df['dataset.parameters.data_name'] == 'US-county-demos' extend the 'dataset.parameters.data_name' with dataset.parameters.task_variable 
# and set it to 'US-county-demos' + '-' + dataset.parameters.task_variable
df.loc[df['dataset.parameters.data_name'] == 'US-county-demos', 'dataset.parameters.data_name'] = df.loc[df['dataset.parameters.data_name'] == 'US-county-demos', 'dataset.parameters.data_name'] + '-' + df.loc[df['dataset.parameters.data_name'] == 'US-county-demos', 'dataset.parameters.task_variable']

In [7]:
# Print all columns 10 per line
for i in range(0, len(df.columns), 5):
    print(list(df.columns[i:i + 5]))


['train/auroc', 'train/accuracy', 'train/recall', 'val/precision', 'trainer/global_step']
['epoch', '_runtime', 'val/auroc', '_timestamp', 'train/loss']
['val/recall', 'train/precision', '_step', 'lr-Adam', 'val/loss']
['val/accuracy', 'seed', 'tags', 'extras', 'trainer']
['ckpt_path', 'task_name', 'model/params/total', 'model/params/trainable', 'model/params/non_trainable']
['_wandb', 'test/precision', 'test/loss', 'test/auroc', 'test/recall']
['test/accuracy', 'val/mae', 'val/mse', 'train/mse', 'train/mae']
['test/mae', 'test/mse', 'epoch_run_time', 'model.compile', 'model._target_']
['model.model_name', 'model.model_domain', 'model.loss.task', 'model.loss._target_', 'model.loss.loss_type']
['model.readout._target_', 'model.readout.hidden_dim', 'model.readout.readout_name', 'model.readout.num_cell_dimensions', 'model.backbone._target_']
['model.backbone.n_layers', 'model.backbone.sc_order', 'model.backbone.aggr_norm', 'model.backbone.conv_order', 'model.backbone.update_func']
['model

### See unique datasets

In [8]:
print(df['dataset.parameters.data_name'].unique())
print("Num unique datasets:", len(df['dataset.parameters.data_name'].unique()))

['roman_empire' 'PROTEINS' 'ZINC' 'NCI109' 'PubMed' 'citeseer' 'Cora'
 'US-county-demos-UnemploymentRate' 'NCI1' 'US-county-demos-BachelorRate'
 nan 'MUTAG' 'US-county-demos-DeathRate' 'US-county-demos-BirthRate'
 'US-county-demos-MigraRate' 'US-county-demos-MedianIncome'
 'US-county-demos-Election']
Num unique datasets: 17


## See unique models

In [9]:
print(df['model.model_name'].unique())

['sccnn_custom' 'scn' 'sccn' nan]


## Solve batch problems

In [11]:
datasets = ['minesweeper', 'roman_empire']
models = ['sccnn_custom', 'scn', 'sccn']
# For the following models and datasets I mistook the batch size, it should be 1, instead of 256 or 128
# Keep the run where batch size is 128 and then change the batch size to 1
for model in models:
    print("MODEL:", model)
    for dataset in datasets:

        # Change the batch size to 1 when it is 128
        
        print(df.loc[(df['model.model_name'] == model) & (df['dataset.parameters.data_name'] == dataset), 'dataset.parameters.batch_size'].unique())
        

MODEL: sccnn_custom
[]
[1.]
MODEL: scn
[]
[]
MODEL: sccn
[]
[]


## Solve issue with projection dropout

In [12]:
print(df['model.feature_encoder.proj_dropout'].unique())

[0.25 0.5   nan]


In [13]:
# Keep rows where model.feature_encoder.proj_dropout is [0.5  0.25]
df = df[df['model.feature_encoder.proj_dropout'].isin([0.5, 0.25])]


In [14]:
df.reset_index(drop=True, inplace=True)

In [15]:
# Sweeped parameters: 
sweeped_columns = [
    'model.optimizer.lr', 
    'model.feature_encoder.out_channels',
    'model.backbone.n_layers',
    'model.readout.readout_name',
    'dataset.transforms.graph2simplicial_lifting.signed',
    'model.feature_encoder.proj_dropout',
    'dataset.parameters.batch_size',
    'dataset.parameters.data_seed',
    'seed',
]



# For each model and dataset go over all the sweeped parameters and print the unique values
for model in df['model.model_name'].unique():
    print(f"Model: {model}")
    for dataset in df['dataset.parameters.data_name'].unique():
        print(f"Dataset: {dataset}")
        for column in sweeped_columns:
            print(f"Column: {column}")
            print(df.loc[(df['model.model_name'] == model) & (df['dataset.parameters.data_name'] == dataset), column].unique())
        
        print('---------------NEW DATASET------------------')
    print('---------------NEW MODEL------------------')


Model: sccnn_custom
Dataset: roman_empire
Column: model.optimizer.lr
[0.001 0.01 ]
Column: model.feature_encoder.out_channels
[ 32. 128.  64.]
Column: model.backbone.n_layers
[2. 1. 4. 3.]
Column: model.readout.readout_name
['PropagateSignalDown' 'NoReadOut']
Column: dataset.transforms.graph2simplicial_lifting.signed
[True]
Column: model.feature_encoder.proj_dropout
[0.25 0.5 ]
Column: dataset.parameters.batch_size
[1.]
Column: dataset.parameters.data_seed
[9. 7. 5. 3. 0.]
Column: seed
[42.]
---------------NEW DATASET------------------
Dataset: PROTEINS
Column: model.optimizer.lr
[0.001 0.01 ]
Column: model.feature_encoder.out_channels
[128.  64.  32.]
Column: model.backbone.n_layers
[4. 3. 2. 1.]
Column: model.readout.readout_name
['PropagateSignalDown' 'NoReadOut']
Column: dataset.transforms.graph2simplicial_lifting.signed
[True]
Column: model.feature_encoder.proj_dropout
[0.5  0.25]
Column: dataset.parameters.batch_size
[256. 128.]
Column: dataset.parameters.data_seed
[9. 7. 5. 3. 0

### Get the best results

In [16]:
# Extract best results for each model and dataset
# 1. Keep the columns that are necessary for the comparison
sweeped_columns = [
    'model.optimizer.lr', 
    'model.feature_encoder.out_channels',
    'model.backbone.n_layers',
    'model.readout.readout_name',
    'dataset.transforms.graph2simplicial_lifting.signed',
    'model.feature_encoder.proj_dropout',
    'dataset.parameters.batch_size',
    # 'dataset.parameters.data_seed',
    # 'seed',
]
run_columns = ['dataset.parameters.data_seed','seed']

# Dataset and model columns
dataset_model_columns = ['model.model_name', 'dataset.parameters.data_name']

# Performance columns
performance_columns = [
    'val/loss', 'test/loss',
    'val/mae', 'test/mae',
    'val/mse', 'test/mse',
    'val/accuracy', 'test/accuracy',
    'val/auroc','test/auroc',
    'val/recall', 'test/recall',
    'val/precision', 'test/precision',
    ]
keep_columns = dataset_model_columns + sweeped_columns + performance_columns + run_columns
df = df[keep_columns]

In [17]:
performance_classification = [
    'val/accuracy', 'test/accuracy',
    'val/auroc','test/auroc',
    'val/recall', 'test/recall',
    'val/precision', 'test/precision',
    ]
performance_regression = [
    'val/mae', 'test/mae',
    'val/mse', 'test/mse',
    ]
# Define a dict of dicts for each dataset the corresponding optimization metrics
optimization_metrics = {
    'IMDB-MULTI': {'optim_metric': 'val/accuracy', 'eval_metric': 'test/accuracy', 'direction': 'max', 'performance_columns': performance_classification},
    'IMDB-BINARY': {'optim_metric': 'val/accuracy', 'eval_metric': 'test/accuracy', 'direction': 'max', 'performance_columns': performance_classification},
    'REDDIT-BINARY': {'optim_metric': 'val/accuracy', 'eval_metric': 'test/accuracy', 'direction': 'max', 'performance_columns': performance_classification},
    'NCI109': {'optim_metric': 'val/accuracy', 'eval_metric': 'test/accuracy', 'direction': 'max', 'performance_columns': performance_classification},
    'NCI1': {'optim_metric': 'val/accuracy', 'eval_metric': 'test/accuracy', 'direction': 'max', 'performance_columns': performance_classification},
    'PROTEINS': {'optim_metric': 'val/accuracy', 'eval_metric': 'test/accuracy', 'direction': 'max', 'performance_columns': performance_classification},
    'MUTAG': {'optim_metric': 'val/accuracy', 'eval_metric': 'test/accuracy', 'direction': 'max', 'performance_columns': performance_classification},
    'Cora': {'optim_metric': 'val/accuracy', 'eval_metric': 'test/accuracy', 'direction': 'max', 'performance_columns': performance_classification},
    'citeseer': {'optim_metric': 'val/accuracy', 'eval_metric': 'test/accuracy', 'direction': 'max', 'performance_columns': performance_classification},
    'PubMed': {'optim_metric': 'val/accuracy', 'eval_metric': 'test/accuracy', 'direction': 'max', 'performance_columns': performance_classification},

    'roman_empire': {'optim_metric': 'val/accuracy', 'eval_metric': 'test/accuracy', 'direction': 'max', 'performance_columns': performance_classification},
    'amazon_ratings': {'optim_metric': 'val/accuracy', 'eval_metric': 'test/accuracy', 'direction': 'max', 'performance_columns': performance_classification},
    
    'tolokers': {'optim_metric': 'val/auroc', 'eval_metric': 'test/auroc', 'direction': 'max', 'performance_columns': performance_classification},
    'questions': {'optim_metric': 'val/auroc', 'eval_metric': 'test/auroc', 'direction': 'max', 'performance_columns': performance_classification},
    'minesweeper': {'optim_metric': 'val/auroc', 'eval_metric': 'test/auroc', 'direction': 'max', 'performance_columns': performance_classification},

    'ZINC': {'optim_metric': 'val/mse', 'eval_metric': 'test/mae', 'direction': 'min', 'performance_columns': performance_regression},
    
    'US-county-demos-UnemploymentRate': {'optim_metric': 'val/mse', 'eval_metric': 'test/mae', 'direction': 'min', 'performance_columns': performance_regression},
    'US-county-demos-BachelorRate': {'optim_metric': 'val/mse', 'eval_metric': 'test/mae', 'direction': 'min', 'performance_columns': performance_regression},
    'US-county-demos-DeathRate': {'optim_metric': 'val/mse', 'eval_metric': 'test/mae', 'direction': 'min', 'performance_columns': performance_regression},
    'US-county-demos-BirthRate': {'optim_metric': 'val/mse', 'eval_metric': 'test/mae', 'direction': 'min', 'performance_columns': performance_regression},
    'US-county-demos-MigraRate': {'optim_metric': 'val/mse', 'eval_metric': 'test/mae', 'direction': 'min', 'performance_columns': performance_regression},
    'US-county-demos-MedianIncome': {'optim_metric': 'val/mse', 'eval_metric': 'test/mae', 'direction': 'min', 'performance_columns': performance_regression},
    'US-county-demos-Election': {'optim_metric': 'val/mse', 'eval_metric': 'test/mae', 'direction': 'min', 'performance_columns': performance_regression},

} 

len(optimization_metrics)

23

### Generate the best results

In [34]:
# Get unique datasets
datasets = list(df['dataset.parameters.data_name'].unique())
# Get unique models
models = list(df['model.model_name'].unique())

best_results = defaultdict(dict)
hp_runs = defaultdict(dict)
best_runs = defaultdict(dict)
# Got over each dataset and model and find the best result
for dataset in datasets:
    for model in models:
        # Get the subset of the DataFrame for the current dataset and model
        subset = df[
            (df['dataset.parameters.data_name'] == dataset)
            & (df['model.model_name'] == model)
        ]

        optim_metric = optimization_metrics[dataset]['optim_metric']
        eval_metric = optimization_metrics[dataset]['eval_metric']
        direction = optimization_metrics[dataset]['direction']
        
        # Keep metrics that matters for dataset
        performance_columns = optimization_metrics[dataset]['performance_columns']
        subset = subset[dataset_model_columns + sweeped_columns + performance_columns + run_columns]

        aggregated = subset.groupby(sweeped_columns).agg(
            {col: ["mean", "std"] for col in performance_columns}
        )

         # Go from MultiIndex to Index
        aggregated = aggregated.reset_index()
        aggregated = aggregated.sort_values(
                by=(optim_metric, "mean"), ascending=(direction == 'min')
            )
        
        # Git percent in case of classification
        if 'test/accuracy' in performance_columns:
            # Go over all the performance columns and multiply by 100
            for col in performance_columns:
                aggregated[(col, "mean")] *= 100
                aggregated[(col, "std")] *= 100
            
            # Round performance columns values up to 2 decimal points
            for col in performance_columns:
                aggregated[(col, "mean")] = aggregated[(col, "mean")].round(2)
                aggregated[(col, "std")] = aggregated[(col, "std")].round(2)
            
            
        else:
            # Round all values up to 4 decimal points
            # Round performance columns values up to 4 decimal points
            for col in performance_columns:
                aggregated[(col, "mean")] = aggregated[(col, "mean")].round(4)
                aggregated[(col, "std")] = aggregated[(col, "std")].round(4)
        
            
        
        # Get the best result
        final_best = aggregated.head(1)
        if final_best[(eval_metric, "mean")].any(): 
            best_results[dataset][model] = {
                "mean": final_best[(eval_metric, "mean")].values[0],
                "std": final_best[(eval_metric, "std")].values[0],
            }

            # Extract best runs: 
            best_params = {}
            for col in sweeped_columns:
                best_params[col] = final_best[(col, '')].item()
            
            hp_runs[dataset][model] = subset.copy()
            
            # Start with the entire DataFrame
            filtered_subset = subset.copy()

            # Iterate over each key-value pair in the best parameters dictionary and filter the DataFrame
            for param, value in best_params.items():
                filtered_subset = filtered_subset[filtered_subset[param] == value]
            best_runs[dataset][model] = filtered_subset
        
        else: 
            best_results[dataset][model] = {
                "mean": np.nan,
                "std": np.nan,
            }

        

        
            
        


In [33]:
best_runs['NCI1']['sccnn_custom']

,model.model_name,dataset.parameters.data_name,model.optimizer.lr,model.feature_encoder.out_channels,model.backbone.n_layers,model.readout.readout_name,dataset.transforms.graph2simplicial_lifting.signed,model.feature_encoder.proj_dropout,dataset.parameters.batch_size,val/accuracy,test/accuracy,val/auroc,test/auroc,val/recall,test/recall,val/precision,test/precision,dataset.parameters.data_seed,seed
2858,sccnn_custom,NCI1,0.01,64.0,2.0,PropagateSignalDown,True,0.25,256.0,0.764362,0.737354,0.829037,0.798823,0.764139,0.737447,0.766510,0.737642,9.0,42.0
2881,sccnn_custom,NCI1,0.01,64.0,2.0,PropagateSignalDown,True,0.25,256.0,0.748783,0.742218,0.803307,0.813284,0.749440,0.742717,0.751107,0.744237,7.0,42.0
2899,sccnn_custom,NCI1,0.01,64.0,2.0,PropagateSignalDown,True,0.25,256.0,0.711782,0.728599,0.785002,0.798330,0.709331,0.728767,0.736966,0.735674,5.0,42.0
2926,sccnn_custom,NCI1,0.01,64.0,2.0,PropagateSignalDown,True,0.25,256.0,0.750730,0.729572,0.804623,0.785845,0.750745,0.729636,0.750762,0.729480,3.0,42.0
2952,sccnn_custom,NCI1,0.01,64.0,2.0,PropagateSignalDown,True,0.25,256.0,0.752678,0.762646,0.826156,0.830282,0.753000,0.763285,0.754196,0.764668,0.0,42.0


In [54]:
pd.set_option("display.max_rows", 1000)
pd.set_option("display.max_columns", 1000)
pd.set_option("display.width", 1000)

COLS = [
    'dataset.parameters.data_seed',
    'dataset.parameters.batch_size',
    'model.backbone.n_layers',
    'model.feature_encoder.out_channels',
    'model.readout.readout_name',
    'model.feature_encoder.proj_dropout',
    'model.optimizer.lr', 
    
]

a = hp_runs['US-county-demos-BirthRate']['scn'].sort_values(by=COLS, ascending=False)[COLS]


In [55]:
for col in COLS:
    print(a[col].value_counts())

dataset.parameters.data_seed
5.0    92
3.0    92
7.0    90
9.0    88
0.0    88
Name: count, dtype: int64
dataset.parameters.batch_size
1.0    450
Name: count, dtype: int64
model.backbone.n_layers
3.0    117
2.0    113
4.0    111
1.0    109
Name: count, dtype: int64
model.feature_encoder.out_channels
128.0    153
32.0     150
64.0     147
Name: count, dtype: int64
model.readout.readout_name
PropagateSignalDown    229
NoReadOut              221
Name: count, dtype: int64
model.feature_encoder.proj_dropout
0.25    232
0.50    218
Name: count, dtype: int64
model.optimizer.lr
0.001    229
0.010    221
Name: count, dtype: int64


In [51]:
for col in COLS:
    print(a[col].value_counts())

dataset.parameters.data_seed
0.0    68
9.0    64
7.0    64
5.0    64
3.0    64
Name: count, dtype: int64
dataset.parameters.batch_size
256.0    162
128.0    162
Name: count, dtype: int64
model.backbone.n_layers
1.0    84
4.0    80
3.0    80
2.0    80
Name: count, dtype: int64
model.feature_encoder.out_channels
64.0     160
32.0     160
128.0      4
Name: count, dtype: int64
model.readout.readout_name
PropagateSignalDown    162
NoReadOut              162
Name: count, dtype: int64
model.feature_encoder.proj_dropout
0.25    164
0.50    160
Name: count, dtype: int64
model.optimizer.lr
0.01    324
Name: count, dtype: int64


## Save obtained best results and best runs

In [27]:
# Convert nested dictionary to DataFrame
nested_dict = dict(best_results)
result_dict = pd.DataFrame.from_dict(
    {
        (i, j): nested_dict[i][j]
        for i in nested_dict
        for j in nested_dict[i].keys()
    },
    orient="index",
)

result_dict["performance"] = result_dict.apply(
    lambda x: f"{x['mean']} ± {x['std']}", axis=1
)
result_dict = result_dict.drop(["mean", "std"], axis=1)

# Reset multiindex
result_dict = result_dict.reset_index()
# rename columns
result_dict.columns = ["Dataset", "Model", "Performance"]

result_dict = result_dict.pivot_table(
    index="Model", columns="Dataset", values="Performance", aggfunc="first"
)

In [28]:
result_dict

Dataset,Cora,MUTAG,NCI1,NCI109,PROTEINS,PubMed,US-county-demos-BachelorRate,US-county-demos-BirthRate,US-county-demos-DeathRate,US-county-demos-Election,US-county-demos-MedianIncome,US-county-demos-MigraRate,US-county-demos-UnemploymentRate,ZINC,citeseer,roman_empire
Model,,,,,,,,,,,,,,,,
sccn,80.86 ± 2.16,nan ± nan,nan ± nan,nan ± nan,nan ± nan,88.37 ± 0.48,0.4378 ± 0.007,0.6416 ± 0.0359,0.5524 ± 0.0282,0.5652 ± 0.018,0.3977 ± 0.021,0.6039 ± 0.038,0.4831 ± 0.0223,nan ± nan,69.6 ± 1.83,nan ± nan
sccnn_custom,82.19 ± 1.07,76.17 ± 6.63,74.01 ± 1.38,75.31 ± 0.47,74.19 ± 2.86,88.18 ± 0.32,0.4215 ± 0.014,0.637 ± 0.0436,0.5349 ± 0.02,0.5512 ± 0.0182,0.3816 ± 0.0157,0.5981 ± 0.0337,0.4778 ± 0.0138,0.4088 ± 0.0047,70.23 ± 2.69,89.15 ± 0.32
scn,82.27 ± 1.34,73.62 ± 6.13,nan ± nan,nan ± nan,75.27 ± 2.14,88.72 ± 0.5,0.4079 ± 0.0124,0.6083 ± 0.0296,0.5219 ± 0.0237,0.5233 ± 0.0273,0.366 ± 0.0216,0.5966 ± 0.0405,0.4523 ± 0.0224,nan ± nan,71.24 ± 1.68,nan ± nan


In [83]:
# Increase the number of allowed rows to display
pd.set_option("display.max_rows", 1000)
pd.set_option("display.max_columns", 1000)
pd.set_option("display.width", 1000)
result_dict.to_csv(f"best_results_graph.csv")

In [80]:
# Convert nested dictionary to DataFrame
dataframes = []
for dataset in best_runs.keys():
    for model in best_runs[dataset].keys():
        df = best_runs[dataset][model].reset_index(drop=True)
        dataframes.append(df)
        


In [84]:
# Create a DataFrame from the list of DataFrames
best_runs_df = pd.concat(dataframes)
best_runs_df.to_csv(f"best_runs_graph.csv")